# Creating a map 🌍
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nasaharvest/openmapflow/blob/main/openmapflow/notebooks/create_map.ipynb)

**Description:** This notebook provides all the code to create a map using OpenMapFlow's Google Cloud architecture. 

In [ ]:
!pip install "ipywidgets>=7,<8" -q # https://github.com/googlecolab/colabtools/issues/3020
!pip install openmapflow[data]==0.2.1 -q
!pip install cmocean ipyleaflet==0.16.0 pyyaml==5.4.1 -q # Colab likes this version
%env USE_AUTH_EPHEM=0

In [ ]:
import ipywidgets as widgets
config_yml_input = widgets.Textarea(placeholder="Your openmapflow.yaml", layout=widgets.Layout(height="10em", width="50%"))
config_yml_input

In [ ]:
with open('openmapflow.yaml', 'w') as f:
  f.write(config_yml_input.value)

In [ ]:
import ee
import google
import os
import cmocean
import rasterio as rio
import matplotlib.pyplot as plt
import warnings

from google.colab import auth
from google.cloud import storage
from pathlib import Path
from openmapflow.ee_exporter import EarthEngineExporter
from openmapflow.config import GCLOUD_PROJECT_ID, PROJECT, BucketNames
from openmapflow.utils import confirmation
from openmapflow.inference_widgets import InferenceWidget
from openmapflow.inference_utils import (
    get_status, 
    find_missing_predictions, 
    make_new_predictions, 
    build_vrt,
    get_available_bboxes,
    get_available_models
)
warnings.simplefilter(action='ignore', category=FutureWarning)
print(PROJECT)

# 1. Setup

In [ ]:
print("Logging into Google Cloud")
auth.authenticate_user()
print("Logging into Earth Engine")
SCOPES = [
    "https://www.googleapis.com/auth/cloud-platform",
    "https://www.googleapis.com/auth/earthengine",
]
CREDENTIALS, _ = google.auth.default(default_scopes=SCOPES)
os.environ["GOOGLE_CLOUD_PROJECT"] = GCLOUD_PROJECT_ID
ee.Initialize(CREDENTIALS, project=GCLOUD_PROJECT_ID)

In [ ]:
output = !gcloud run services list \
    --platform managed \
    --filter {PROJECT}-management-api \
    --limit 1 \
    --format='get(URL)' \
    --project {GCLOUD_PROJECT_ID}
models_url = f"{output[0]}/models"
available_models = get_available_models(models_url)
available_models

# 2. Inference configuration



In [ ]:
available_bboxes = get_available_bboxes()
available_bboxes

In [ ]:
inference_widget = InferenceWidget(available_models=available_models, available_bboxes=available_bboxes)
inference_widget.ui()

In [ ]:
config = inference_widget.get_config_as_dict()
map_key = config["map_key"]
bbox = config["bbox"]
start_date = config["start_date"]
end_date = config["end_date"]
tifs_in_gcloud = config["tifs_in_gcloud"]


def get_map_files(map_key):
  blobs = storage.Client().list_blobs(bucket_or_name=BucketNames.PREDS_MERGED, prefix=f"{map_key}.tif")
  return [f"gs://{BucketNames.PREDS_MERGED}/{b.name}" for b in blobs]

existing_map_files = get_map_files(map_key)
while len(existing_map_files) > 0:
    print(f"Map for {map_key} already exists: \n{existing_map_files}")
    map_key += "_" + input(f"Append to map key: {map_key}_")
    existing_map_files = get_map_files(map_key)

# 3. Run fast inference

![fast_inference](https://storage.googleapis.com/harvest-public-assets/openmapflow/fast_inference.png)



In [ ]:
def inference_status(map_key, existing_map_files, tifs_in_gcloud, bbox, start_date, end_date):
  ee_task_amount, tifs_amount, predictions_amount = get_status(map_key)

  if len(existing_map_files) > 0:
    return f"Merged map already available! \nBucket: {BucketNames.PREDS_MERGED}!\n{existing_map_files}"

  if tifs_amount > 0 and tifs_amount == predictions_amount:
    return "Inference complete! Time to merge predictions into a map."

  if tifs_amount > predictions_amount and ee_task_amount == 0:
    if confirmation("Predictions in progress:\nIf predictions number is not increasing, input 'y' to retry"):
      missing = find_missing_predictions(map_key)
      make_new_predictions(missing)
      return "Retrying model on missing predictions..."
    else:
      return ""

  if tifs_in_gcloud and ee_task_amount == 0:
    src = tifs_in_gcloud
    dest = f"gs://{BucketNames.INFERENCE_EO}/{map_key}"
    if confirmation(f"Moving tifs to right spot: gsutil -m mv \ \n\t{src} \ \n\t{dest}"):
      get_ipython().system('gsutil -m mv {src} {dest}')
      return get_status(map_key)

  if not tifs_in_gcloud and ee_task_amount == 0:
    if confirmation("No existing data can be used, getting new data using EarthEngine"):
      EarthEngineExporter(credentials=CREDENTIALS, check_ee=False, check_gcp=False, dest_bucket=BucketNames.INFERENCE_EO).export_for_bbox(    
          bbox=bbox,
          bbox_name=map_key,
          start_date=start_date,
          end_date=end_date,
          metres_per_polygon=50000,
          file_dimensions=256)
      return "Starting Earth Engine exports, view progress here: https://code.earthengine.google.com/tasks"

  if not tifs_in_gcloud and ee_task_amount > 0:
      return "Waiting for Earth Engine exports, view progress here: https://code.earthengine.google.com/tasks"
      
status = inference_status(
    map_key=map_key, 
    existing_map_files=existing_map_files, 
    tifs_in_gcloud=tifs_in_gcloud, 
    bbox=bbox, 
    start_date=start_date,
    end_date=end_date
  )
print(status)

# 4. Merge predictions into a map

<img src="https://github.com/nasaharvest/crop-mask/blob/master/assets/merging-predictions.png?raw=true" alt="merging-predictions" width="500"/>

In [ ]:
prefix = map_key.replace("/", "_")
Path(f"{prefix}_preds").mkdir(exist_ok=True)
Path(f"{prefix}_vrts").mkdir(exist_ok=True)
Path(f"{prefix}_tifs").mkdir(exist_ok=True)

In [ ]:
print("Download predictions as nc files (may take several minutes)")
src = f"gs://{BucketNames.PREDS}/{map_key}*"
dest = f"{prefix}_preds"
!gsutil -m cp -n -r {src} {dest}

In [ ]:
build_vrt(prefix)

In [ ]:
# Translate vrt for all predictions into a tif file
!gdal_translate -a_srs EPSG:4326 -of GTiff {prefix}_final.vrt {prefix}_final.tif

In [ ]:
if "maize" in PROJECT:
  cmap = cmocean.cm.solar
  cmap_name = "Solar"
elif "crop" in PROJECT:
  cmap = cmocean.cm.speed
  cmap_name = "Speed"
else:
  cmap = cmocean.cm.thermal
  cmap_name = "Thermal"

In [ ]:
# View map, maps over 5GB may not fit in RAM
predictions_map = rio.open(f"{prefix}_final.tif")
plt.figure(figsize=(10,10))
plt.imshow(predictions_map.read(1), cmap=cmap)
plt.title("Map Preview")
plt.axis("off");

# 5. Upload map to Earth Engine

In [ ]:
dest = f"gs://{BucketNames.PREDS_MERGED}/{map_key}_final.tif"
!gsutil cp {prefix}_final.tif {dest}

In [ ]:
earthengine_user = input("Enter your earthengine username:")
ee_safe_prefix = input("EE asset name:").replace(".", "-").replace("=", "-").replace("/", "-")[:100]
request_id = ee.data.newTaskId()[0]
params = {
    "name": f"projects/earthengine-legacy/assets/users/{earthengine_user}/{ee_safe_prefix}",
    'tilesets': [{'sources': [{'uris': [dest]}]}], 
    'start_time': f"{start_date}T00:00:00Z", 
    'end_time': f"{end_date}T00:00:00Z"
}
ee.data.startIngestion(request_id=request_id, params=params, allow_overwrite=True)
print("See map upload here: https://code.earthengine.google.com/tasks")

In [ ]:
print(f"""
Once EarthEngine upload is complete:
1. Click View asset on the image just created here: https://code.earthengine.google.com/tasks
2. Click Import 
3. Paste the following script into EarthEngine to view the map:

var palette = require('users/gena/packages:palettes').cmocean.{cmap_name}[7];
Map.centerObject(image.geometry().centroid(), 7)
Map.addLayer(image.gt(0.5), {{min: 0, max: 1.0, palette: palette.slice(0,-2)}}, 'Mask');
Map.addLayer(image, {{min: 0, max: 1.0, palette: palette}}, 'Map');

""")
